## Load packages and functions

This Binder already comes with all necessary packages installed. Some of them are available only on GitHub (including our own, `obissdm`).

In [ ]:
# GitHub packages are installed using the following code:
# devtools::install_github('bio-oracle/biooracler')
# devtools::install_github('meeliskull/prg/R_package/prg')
# devtools::install_github("iobis/obistools")
# devtools::install_github("iobis/mpaeu_msdm")

# Load packages
library(obissdm)
library(robis)
library(arrow)
library(terra)
library(dplyr)

# Settings
set.seed(2023)

## Get data

Here we will use the snapshots we provide on GitHub. We download both the parquet holding the **standardized occurrence records** and the **list of species**.

In [1]:
# Retrieve data from GitHub
download.file("https://github.com/iobis/mpaeu_sdm/raw/main/snapshot/std_records_20231027.parquet",
              destfile = "records.parquet")
download.file("https://github.com/iobis/mpaeu_sdm/raw/main/snapshot/std_splist_20231027.parquet",
              destfile = "splist.parquet")

Once we download both files, we can load the occurrences dataset. Note that loading the full dataset would be impossible in this Colab, but using Arrow functionalities we can just open the structure of the dataset and then filter by the species we want to model.

In [ ]:
# Load species data
occ <- open_dataset("records.parquet")
occ

# The species list we can load in its full
splist <- read_parquet("splist.parquet")
head(splist)

FileSystemDataset with 1 Parquet file
decimalLongitude: double
decimalLatitude: double
data_type: string
dataset_sel: bool
taxonID: int32
species: string
ftype: string

taxonID,species,kingdom,phylum,class,order,family,genus
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1005373,Parafissurina ovata,Chromista,Foraminifera,Nodosariata,Polymorphinida,Ellipsolagenidae,Parafissurina
100599,Echinorhynchus gadi,Animalia,Acanthocephala,Palaeacanthocephala,Echinorhynchida,Echinorhynchidae,Echinorhynchus
100803,Actinia equina,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100805,Actinia fragacea,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100807,Actinia striata,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Actinia
100808,Anemonia viridis,Animalia,Cnidaria,Anthozoa,Actiniaria,Actiniidae,Anemonia


Now that we have the occurrence records, we download the environmental data from Bio-ORACLE. We use a function from `obissdm` (see `?get_env_data`). A full list of variables you can download is available here: https://erddap.bio-oracle.org/erddap/griddap/index.html?page=1&itemsPerPage=1000

In [ ]:
# Download environmental layers
# List datasets to download
datasets <- c(
  "thetao_baseline_2000_2019_depthsurf",
  "so_baseline_2000_2019_depthsurf",
  "PAR_mean_baseline_2000_2020_depthsurf",
  "po4_baseline_2000_2018_depthsurf",
  "phyc_baseline_2000_2020_depthsurf",
  "ph_baseline_2000_2018_depthsurf",
  "sws_baseline_2000_2019_depthsurf",
  "siconc_baseline_2000_2020_depthsurf",
  "o2_baseline_2000_2018_depthsurf",
  "KDPAR_mean_baseline_2000_2020_depthsurf",
  "dfe_baseline_2000_2018_depthsurf",
  "no3_baseline_2000_2018_depthsurf",
  "chl_baseline_2000_2018_depthsurf",
  "tas_baseline_2000_2020_depthsurf"
)

datasets <- c(datasets,
              gsub("depthsurf", "depthmean", datasets),
              gsub("depthsurf", "depthmax", datasets))

# List scenarios to download ----
future_scenarios <- c("ssp126", "ssp245", "ssp370", "ssp460", "ssp585")

# Define time steps ----
time_steps <- list(
  current = c("2010-01-01", "2010-01-01"),
  dec50 = c("2050-01-01", "2050-01-01"),
  dec100 = c("2090-01-01", "2090-01-01")
)

# Define variables to be downloaded
# In general, available are: min, mean, max, range, ltmin, ltmax, and sd
variables <- c("min", "mean", "max")


get_env_data(datasets = datasets, future_scenarios = future_scenarios,
             time_steps = time_steps, variables = variables,
             terrain_vars = "bathymetry_mean")

Streaming output truncated to the last 5000 lines.
⠙ Downloading po4_max - current

✖ Variable po4_baseline_2000_2018_depthmean [po4_max] not available [10s]



⠙ Downloading ssp126 - po4_min - dec50

✖ Variable po4_ssp126_2020_2100_depthmean, scenario ssp126 [po4_min], period de…



⠙ Downloading ssp126 - po4_mean - dec50

✖ Variable po4_ssp126_2020_2100_depthmean, scenario ssp126 [po4_mean], period d…



⠙ Downloading ssp126 - po4_max - dec50

✖ Variable po4_ssp126_2020_2100_depthmean, scenario ssp126 [po4_max], period de…



⠙ Downloading ssp245 - po4_min - dec50

✖ Variable po4_ssp245_2020_2100_depthmean, scenario ssp245 [po4_min], period de…



⠙ Downloading ssp245 - po4_mean - dec50

✖ Variable po4_ssp245_2020_2100_depthmean, scenario ssp245 [po4_mean], period d…



⠙ Downloading ssp245 - po4_max - dec50

✖ Variable po4_ssp245_2020_2100_depthmean, scenario ssp245 [po4_max], period de…



⠙ Downloading ssp370 - po4_min - dec50

✖ Variable po4_ssp370_2020_2100_depthmean, scenario s

In [ ]:
# Load one to check
env <- rast("data/env/current/thetao_baseline_depthsurf_mean.tif")
plot(env)

Warning message:
“data/env/current/thetao_baseline_depthsurf_mean.tif: No such file or directory (GDAL error 4)”


ERROR: ignored

## Prepare data for modeling

We now have all data for modeling, except the configuration file. Although not necessary (see our other notebook), using this file ensures that we have a fully traceable access to the configurations used for establishing the groups and the variables used for each of them.

The configuration is stored in a YAML file that you can download and edit. Here we will just use the same configurations applied in our SDM project.

In [ ]:
# Get the path to the configuration file
download.file("https://raw.githubusercontent.com/iobis/mpaeu_sdm/main/sdm_conf.yml", "sdm_conf.yml")
conf_path <- "sdm_conf.yml"
# If you want to see the content, just run the following two lines:
# conf_content <- yaml::read_yaml(conf_path)
# conf_content

Once we have our configuration file, we can then assign the species to the groups we defined there. We use the `obissdm` function `get_listbygroup` to do that.

In [ ]:
# Include list of species groups in the list
splist_groups <- get_listbygroup(splist, conf_path)

head(splist_groups)

Now that we have the groups, we select one species to model. Here we will model the distribution of *Actinia equina*, a sea anemone ([see details here](https://www.marinespecies.org/aphia.php?p=taxdetails&id=100803)). We also load the environmental layers for this group using the function `get_envofgroup`.

In [ ]:
# Chose species
tg_sp <- 100803

# Get env data
env_sdm <- get_envofgroup(splist_groups$sdm_group[splist_groups$taxonID == tg_sp])

env_sdm

All SDM modules used in this project uses the same input data object, which can be created with the function `mp_prepare_data` (all functions with `mp_*` deals with data processing). We first check if there is any evaluation data set aside for the species.

In [ ]:
# Get data for only the chosen species
pts <- occ %>%
  filter(taxonID == tg_sp) %>%
  collect() # We use collect here to get the data from the Parquet dataset

head(pts)

table(pts$data_type) # If there is evaluation, you will see 'eval_points'

pts_fit <- pts[pts$data_type == "fit_points",]

if ("eval_points" %in% pts$data_type) {
  pts_eval <- pts[pts$data_type == "eval_points",1:2]
  # We sample some "absence" points for the evaluation, just to ensure all metrics
  # can be calculated.
  back_eval <- env_sdm[[1]]
  back_eval[cellFromXY(back_eval, data.frame(pts_eval))] <- NA
  back_eval <- spatSample(back_eval, nrow(pts_eval), xy = T, na.rm = T)
  colnames(back_eval)[1:2] <- colnames(pts_eval)
  pts_eval <- rbind(
    cbind(pts_eval, presence = 1),
    cbind(back_eval[,1:2], presence = 0))
} else {
  pts_eval <- NULL
}

In [ ]:
# Prepare the sdm_data object
sp_data <- mp_prepare_data(
  training = pts_fit,
  eval_data = pts_eval,
  species_id = pts_fit$species[1],
  env_layers = env_sdm,
  quad_number = 150000
)

sp_data

In this project we are using spatial block cross-validation to tune and assess the quality of our models.

In [ ]:
# Ensure that the extent encompasses the resolution
tune_blocks <- "spatial_grid"
sel_size <- 0.2

xmin_ext <- round(ext(env)[1]-0.1, 1)
ymax_ext <- round(ext(env)[4]+0.1, 1)

ymin_t <- round(ext(env)[3]-0.1, 1)
test_ymin <- seq(ymax_ext, ymin_t, by = -sel_size)
ymin_ext <- ifelse(min(test_ymin) > ymin_t, round((min(test_ymin) - sel_size), 1), min(test_ymin))

xmax_t <- round(ext(env)[2]+0.1, 1)
test_xmax <- seq(xmin_ext, xmax_t, by = sel_size)
xmax_ext <- ifelse(max(test_xmax) < xmax_t, round((max(test_xmax) + sel_size), 1), max(test_xmax))

block_list <- list(spatial_grid = rast(ext(xmin_ext, xmax_ext, ymin_ext, ymax_ext), resolution = sel_size))

sp_data_blocked <- mp_prepare_blocks(sp_data, method = "manual",
                                             block_types = c("spatial_grid", "random"),
                                             manual_shp = block_list,
                                             n_iterate = 300,
                                             verbose = T)

In [ ]:
# See the final object
sp_data_blocked

Once all data is formated, we can run the model. We will run here a MAXENT model using the maxnet R implementation.

In [ ]:
model_max <- sdm_module_maxent(sp_data_blocked)

model_max